In [ ]:
%pip install nba_api
%pip install pandas
# https://github.com/swar/nba_api

In [ ]:
from nba_api.stats.static import teams
allTeams = teams.get_teams()
teamValues = {}
teamsPlayed = {}
teamIDs = []
values = {}
abbreviation_to_id = {}
id_to_abbreviation = {}

for i in range(len(allTeams)):
  # WP, OWP, OOWP, RPI
  teamValues[allTeams[i]['id']] = [0, 0, 0, 0]
  # Teams that the current team has played
  teamsPlayed[allTeams[i]['id']] = []
  teamIDs.append(allTeams[i]['id'])
  values[allTeams[i]['id']] = []
  abbreviation_to_id[allTeams[i]['abbreviation']] = allTeams[i]['id']
  id_to_abbreviation[allTeams[i]['id']] = allTeams[i]['abbreviation']

print(abbreviation_to_id)

{'ATL': 1610612737, 'BOS': 1610612738, 'CLE': 1610612739, 'NOP': 1610612740, 'CHI': 1610612741, 'DAL': 1610612742, 'DEN': 1610612743, 'GSW': 1610612744, 'HOU': 1610612745, 'LAC': 1610612746, 'LAL': 1610612747, 'MIA': 1610612748, 'MIL': 1610612749, 'MIN': 1610612750, 'BKN': 1610612751, 'NYK': 1610612752, 'ORL': 1610612753, 'IND': 1610612754, 'PHI': 1610612755, 'PHX': 1610612756, 'POR': 1610612757, 'SAC': 1610612758, 'SAS': 1610612759, 'OKC': 1610612760, 'TOR': 1610612761, 'UTA': 1610612762, 'MEM': 1610612763, 'WAS': 1610612764, 'DET': 1610612765, 'CHA': 1610612766}


In [ ]:
from nba_api.stats.endpoints import leaguestandingsv3
from nba_api.stats.endpoints import leaguegamefinder

standings = leaguestandingsv3.LeagueStandingsV3()

for i in range(30):
  id = standings.standings.data['data'][i][2]

  # home record
  home = standings.standings.data['data'][i][18]
  # away record
  away = standings.standings.data['data'][i][19]
  # wins
  wins = standings.standings.data['data'][i][13]
  # loses
  loses = standings.standings.data['data'][i][14]
  total_games = wins + loses

  # Query for games where the Celtics were playing
  gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=id)
  # The first DataFrame of those returned is what we want.
  games = gamefinder.get_data_frames()[0][:total_games]

  home_wins = int(home.split('-')[0]) * 0.6
  home_losses = int(home.split('-')[1]) * 1.4

  away_wins = int(away.split('-')[0]) * 1.4
  away_losses = int(away.split('-')[1]) * 0.6

  weighted_WP = (home_wins + away_wins) - (home_losses + away_losses)
  WP = wins/total_games
  values[id].append(weighted_WP)
  values[id].append(WP)

  for i in range(len(games['MATCHUP'])):
    opp = ''
    if ' vs.' in games['MATCHUP'][i]:
      opp = games['MATCHUP'][i].split( 'vs. ')[1]
    elif ' @ ' in games['MATCHUP'][i]:
      opp = games['MATCHUP'][i].split( '@ ')[1]
    opp_id = abbreviation_to_id[opp]
    teamsPlayed[id].append(opp_id)

values

{1610612737: [-6.399999999999999, 0.39473684210526316],
 1610612738: [21.4, 0.7692307692307693],
 1610612739: [5.0, 0.5945945945945946],
 1610612740: [7.9999999999999964, 0.6],
 1610612741: [-5.0, 0.4634146341463415],
 1610612742: [5.199999999999999, 0.575],
 1610612743: [14.599999999999998, 0.6829268292682927],
 1610612744: [-5.0, 0.46153846153846156],
 1610612745: [-1.5999999999999996, 0.5],
 1610612746: [10.6, 0.6410256410256411],
 1610612747: [-2.0, 0.475],
 1610612748: [7.399999999999999, 0.5897435897435898],
 1610612749: [13.600000000000001, 0.7],
 1610612750: [18.2, 0.717948717948718],
 1610612751: [-6.0, 0.42105263157894735],
 1610612752: [10.6, 0.5897435897435898],
 1610612753: [4.199999999999999, 0.5384615384615384],
 1610612754: [5.800000000000001, 0.5897435897435898],
 1610612755: [9.8, 0.6486486486486487],
 1610612756: [1.0000000000000036, 0.5384615384615384],
 1610612757: [-16.999999999999996, 0.2564102564102564],
 1610612758: [6.600000000000001, 0.5897435897435898],
 161

In [ ]:
# https://squared2020.com/2017/03/10/how-to-game-the-rating-percentage-index-rpi-in-basketball/
# RPI = 0.25*WP + 0.50*OWP + 0.25*OOWP

OWP_per_team = {}

for id in teamIDs:
  total = 0
  count = 0
  for opp_id in teamsPlayed[id]:
    total += values[opp_id][1]
    count += 1

  OWP_per_team[id] = (total/count)

OWP_per_team

{1610612737: 0.4933159448732722,
 1610612738: 0.514590580318338,
 1610612739: 0.4828381755263067,
 1610612740: 0.5164425034049551,
 1610612741: 0.4981756142171308,
 1610612742: 0.48599263786844027,
 1610612743: 0.4886049071973803,
 1610612744: 0.5322225950570969,
 1610612745: 0.5145382652873703,
 1610612746: 0.4961178839548546,
 1610612747: 0.513386399260918,
 1610612748: 0.4759732261904672,
 1610612749: 0.46596632174536534,
 1610612750: 0.5294027635811975,
 1610612751: 0.4977185845733579,
 1610612752: 0.5043033778246576,
 1610612753: 0.5154293812311982,
 1610612754: 0.5048361456305566,
 1610612755: 0.46126830535375785,
 1610612756: 0.4523820424704201,
 1610612757: 0.5088907636700665,
 1610612758: 0.498301762237281,
 1610612759: 0.5241944352713896,
 1610612760: 0.5007254156002551,
 1610612761: 0.4958448391498653,
 1610612762: 0.5025289176828049,
 1610612763: 0.5176219702091134,
 1610612764: 0.49699406150237174,
 1610612765: 0.5117055112932472,
 1610612766: 0.5072132961787058}

In [ ]:
OOWP_per_team = {}

for id in teamIDs:
  total = 0
  count = 0
  for opp_id in teamsPlayed[id]:
    total += OWP_per_team[opp_id]
    count += 1
  OOWP_per_team[id] = (total/count)

OOWP_per_team

{1610612737: 0.4972732990298614,
 1610612738: 0.4996089559710103,
 1610612739: 0.5020135384690892,
 1610612740: 0.5044652878603195,
 1610612741: 0.4956607558374591,
 1610612742: 0.506664042626337,
 1610612743: 0.5063890875539065,
 1610612744: 0.49785968874987463,
 1610612745: 0.49981375144151446,
 1610612746: 0.5049911196834304,
 1610612747: 0.49942905332601006,
 1610612748: 0.5012846117942715,
 1610612749: 0.5023946798764756,
 1610612750: 0.5023373170995127,
 1610612751: 0.4959175277211181,
 1610612752: 0.49635790128325213,
 1610612753: 0.49686725490004263,
 1610612754: 0.4964813187357945,
 1610612755: 0.5028531851057382,
 1610612756: 0.5069654058556947,
 1610612757: 0.49769547572471196,
 1610612758: 0.5042280756481462,
 1610612759: 0.5001350664282074,
 1610612760: 0.5033907425674468,
 1610612761: 0.49730088710532694,
 1610612762: 0.49950330671958243,
 1610612763: 0.5003986869860819,
 1610612764: 0.4959319830317866,
 1610612765: 0.4970111475208314,
 1610612766: 0.496423176186107}

In [ ]:
RPI_per_team = {}
for id in teamIDs:
  RPI = 0.25*values[id][0] + 0.5*OWP_per_team[id] + 0.25*OOWP_per_team[id]
  RPI_per_team[id_to_abbreviation[id]] = RPI

power_rankings = sorted(RPI_per_team.items(), key=lambda x:x[1])
print(power_rankings)

[('DET', -7.769894457473168), ('SAS', -5.612869015757253), ('WAS', -5.027519973490867), ('CHA', -4.57228755786412), ('POR', -3.871130749233788), ('MEM', -2.0660893431489233), ('TOR', -1.5777523586487368), ('ATL', -1.2290237028058983), ('BKN', -1.1271613257830415), ('CHI', -0.8769970039320698), ('GSW', -0.8594237802839829), ('LAL', -0.11844953703803848), ('HOU', -0.017777429495936123), ('PHX', 0.6029323726991347), ('UTA', 0.9261402855212979), ('ORL', 1.4319315043406096), ('CLE', 1.6169224723804256), ('DAL', 1.6696623295908042), ('IND', 1.826538402499227), ('SAC', 2.0252079000306775), ('MIA', 2.213307766043801), ('NOP', 2.3843375736675565), ('PHI', 2.806347448953314), ('LAC', 3.024306721898285), ('NYK', 3.026241164233142), ('MIL', 3.758581830841802), ('OKC', 3.9762103934419892), ('DEN', 4.020899725487166), ('MIN', 4.940285711065477), ('BOS', 5.732197529151921)]
